# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [15]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)



# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


In [16]:
#Here I am finding all of the things we were asked to find for the district summary. 
#Some of them are clunky and awkward, but I believe they all work.

school_count = school_data_complete.school_name.nunique()
total_budget = school_data.budget.sum()
total_students = school_data["size"].sum()
avg_math = student_data.math_score.mean()
avg_read = student_data.reading_score.mean()
math_pass = 100*(sum(i>=70 for i in student_data.math_score) / total_students)
read_pass = 100*(sum(i>=70 for i in student_data.reading_score) / total_students)
stu_avg = (student_data["math_score"]+student_data["reading_score"])/2
overall_pass = (math_pass+read_pass)/2



In [17]:
#Here I create a dataframe that uses the values computed above

summary = [{"Total Schools": school_count, 
           "Total Students": total_students,
           "Total Budget": total_budget,
          "Average Math Score": avg_math,
          "Average Reading Score": avg_read,
          "% Passing Math": math_pass,
          "% Passing Reading": read_pass,
           "% Overall Passing Rate": overall_pass}]

district_summary_df = pd.DataFrame(summary)
district_summary_df_org = district_summary_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score",
                             "% Passing Math","% Passing Reading", "% Overall Passing Rate"]]
district_summary_df_org

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [18]:
#Here I am computing all the quatities I need for the school summary.  

index = [i for i in school_data.school_name]
school_type = [i for i in school_data.type]
school_size =  [float(i) for i in school_data["size"]]
school_budget =  [float(i) for i in school_data["budget"]]

#average amount spent per student

school_per_student = []
for i in range(len(school_size)):
    l=float(school_budget[i]/school_size[i])
    school_per_student.append(l)
school_per_student

#this monster is finding the math averages in the right order

i=pd.DataFrame(index)
i.columns=['school_name']
s=pd.DataFrame(student_data.groupby("school_name").math_score.mean())
math_mean = pd.merge(i, s, how="left", on=["school_name", "school_name"])
math_mean.head()
school_avg_math = [i for i in math_mean.math_score]

#this monster is finding the reading averages in the right order

i=pd.DataFrame(index)
i.columns=['school_name']
s=pd.DataFrame(student_data.groupby("school_name").reading_score.mean())
reading_mean = pd.merge(i, s, how="left", on=["school_name", "school_name"])
reading_mean.head()
school_avg_reading = [i for i in reading_mean.reading_score]

#This thing find the percentage of students with a passing math grade

pass_math = student_data.loc[student_data["math_score"] >= 70, ["school_name","math_score"]]
s=pd.DataFrame(pass_math.groupby("school_name").count())
i=pd.DataFrame(index)
i.columns=['school_name']
pass_math_df = pd.merge(i, s, how="left", on=["school_name", "school_name"])
pass_math_order = [i for i in pass_math_df.math_score]
pass_math_order

math_pass_percent = []
for i in range(len(school_size)):
    l=float(100*pass_math_order[i]/school_size[i])
    math_pass_percent.append(l)
math_pass_percent


#This thing find the percentage of students with a passing reading grade

pass_reading = student_data.loc[student_data["reading_score"] >= 70, ["school_name","reading_score"]]
s=pd.DataFrame(pass_reading.groupby("school_name").count())
i=pd.DataFrame(index)
i.columns=['school_name']
pass_reading_df = pd.merge(i, s, how="left", on=["school_name", "school_name"])
pass_reading_order = [i for i in pass_reading_df.reading_score]
pass_reading_order
reading_pass_percent = []
for i in range(len(school_size)):
    l=float(100*pass_reading_order[i]/school_size[i])
    reading_pass_percent.append(l)
reading_pass_percent

#The average of the two lists

avg_pass = []
for i in range(len(school_size)):
    avg_pass.append((math_pass_percent[i]+reading_pass_percent[i])/2)


In [19]:
school_summary = pd.DataFrame({"School Type": school_type,
                              "Total Student": school_size,
                              "Total Budget":  school_budget,
                              "Per Student Budget":school_per_student ,
                              "Average Math Score": school_avg_math ,
                              "Average Reading Score": school_avg_reading ,
                              "% Passing Math": math_pass_percent ,
                              "% Passing Reading":reading_pass_percent ,
                              "Overall Pass Rate": avg_pass}, index = index)
school_summary

,School Type,Total Student,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761.0,1056600.0,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


## School Summary

* Sort and display the top five schools in overall passing rate

In [20]:
#Here is am displaying the top 5 schools based on overall passing rate

top_pass_rate=school_summary.sort_values("Overall Pass Rate", ascending=False)
top_pass_rate.head(5)

,School Type,Total Student,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [21]:
#Here is am displaying the bottom 5 schools based on overall passing rate

bottom_pass_rate=school_summary.sort_values("Overall Pass Rate")
bottom_pass_rate.head(5)

,School Type,Total Student,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [22]:
#After doing this it is abundantly clear that making a function would have been way easier,
#but at the moment I am just glad that something is working.

#math_score_df will be the final data frame.  I will append columns one at a time

math_score_df = pd.DataFrame(index)
math_score_df.set_index(0)
math_score_df.columns=['school_name']

#appending the 9th grade column

math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
ninth_math = math_grade.loc[math_grade['grade']=="9th", ["school_name","math_score"]]
ninth_math=ninth_math.set_index("school_name")
ninth_math = ninth_math.rename(columns = {"math_score":"9th Grade"} )
math_score_df = pd.merge(math_score_df, ninth_math, how="left", on=["school_name", "school_name"])
ninth_math

#appending the 10th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
tenth_math = math_grade.loc[math_grade['grade']=="10th", ["school_name","math_score"]]
tenth_math=tenth_math.set_index("school_name")
tenth_math = tenth_math.rename(columns = {"math_score":"10th Grade"} )
math_score_df= pd.merge(math_score_df, tenth_math, how="left", on=["school_name", "school_name"])
math_score_df

#appending the 11th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
eleventh_math = math_grade.loc[math_grade['grade']=="11th", ["school_name","math_score"]]
eleventh_math=eleventh_math.set_index("school_name")
eleventh_math = eleventh_math.rename(columns = {"math_score":"11th Grade"} )
math_score_df= pd.merge(math_score_df, eleventh_math, how="left", on=["school_name", "school_name"])
math_score_df


#appending the 12th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
twelfth_math = math_grade.loc[math_grade['grade']=="12th", ["school_name","math_score"]]
twelfth_math= twelfth_math.set_index("school_name")
twelfth_math = twelfth_math.rename(columns = {"math_score":"12th Grade"} )
math_score_df= pd.merge(math_score_df,twelfth_math, how="left", on=["school_name", "school_name"])
math_score_df.set_index('school_name')


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Huang High School,77.027251,75.908735,76.446602,77.225641
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Shelton High School,83.420755,82.917411,83.383495,83.778976
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Griffin High School,82.044010,84.229064,83.842105,83.356164
Wilson High School,83.085578,83.724422,83.195326,83.035794
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Bailey High School,77.083676,76.996772,77.515588,76.492218
Holden High School,83.787402,83.429825,85.000000,82.855422


## Reading Score by Grade 

In [23]:
#reading_score_df will be the final data frame.  I will append columns one at a time

#Wow writing my own functions would have been way smarter.

reading_score_df = pd.DataFrame(index)
reading_score_df.set_index(0)
reading_score_df.columns=['school_name']

#appending the 9th grade column

math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
ninth_math = math_grade.loc[math_grade['grade']=="9th", ["school_name","reading_score"]]
ninth_math=ninth_math.set_index("school_name")
ninth_math = ninth_math.rename(columns = {"reading_score":"9th Grade"} )
reading_score_df = pd.merge(reading_score_df, ninth_math, how="left", on=["school_name", "school_name"])


#appending the 10th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
tenth_math = math_grade.loc[math_grade['grade']=="10th", ["school_name","reading_score"]]
tenth_math=tenth_math.set_index("school_name")
tenth_math = tenth_math.rename(columns = {"reading_score":"10th Grade"} )
reading_score_df= pd.merge(reading_score_df, tenth_math, how="left", on=["school_name", "school_name"])


#appending the 11th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
eleventh_math = math_grade.loc[math_grade['grade']=="11th", ["school_name","reading_score"]]
eleventh_math=eleventh_math.set_index("school_name")
eleventh_math = eleventh_math.rename(columns = {"reading_score":"11th Grade"} )
reading_score_df= pd.merge(reading_score_df, eleventh_math, how="left", on=["school_name", "school_name"])



#appending the 12th grade column
math_grade=pd.DataFrame(student_data.groupby(['school_name','grade']).mean())
math_grade = math_grade.reset_index()
twelfth_math = math_grade.loc[math_grade['grade']=="12th", ["school_name","reading_score"]]
twelfth_math= twelfth_math.set_index("school_name")
twelfth_math = twelfth_math.rename(columns = {"reading_score":"12th Grade"} )
reading_score_df= pd.merge(reading_score_df,twelfth_math, how="left", on=["school_name", "school_name"])
reading_score_df.set_index('school_name')


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Huang High School,81.290284,81.512386,81.417476,80.305983
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Shelton High School,84.122642,83.441964,84.373786,82.781671
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Griffin High School,83.369193,83.706897,84.288089,84.013699
Wilson High School,83.939778,84.021452,83.764608,84.317673
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Bailey High School,81.303155,80.907183,80.945643,80.912451
Holden High School,83.677165,83.324561,83.815534,84.698795


## Scores by School Spending

In [24]:
# I think this went much better than above

spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary["bin_group"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)
school_summary.head()

school_bins = pd.DataFrame(school_summary.groupby('bin_group')["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].mean())
school_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
bin_group,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [25]:
# I think this went much better than above


size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary["size_bin_group"] = pd.cut(school_summary["Total Student"], size_bins, labels=group_names)
school_summary.head()

school_bins = pd.DataFrame(school_summary.groupby('size_bin_group')["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].mean())
school_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
size_bin_group,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [12]:
# I think this went much better than above

type_group = school_summary.groupby("School Type")["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].mean()
type_group

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
